## Тут на большом корпусе док-в

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import os
import re
import xgboost

from numpy import loadtxt
from xgboost import XGBClassifier
import time
import pickle

In [2]:
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
import gensim
import pymorphy2
import numpy as np
import pandas as pd

from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn import preprocessing

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.tree import DecisionTreeClassifier

In [3]:
import tqdm

In [4]:
import re

In [5]:
def read_txt(path,class_of_t):
    for root, dirs, files in os.walk(path):
        for filename in files:
            open_name = os.path.join(root, filename)
            f = open(open_name)
            try:
                cleantext = BeautifulSoup(f, 'lxml').text
                yield class_of_t, cleantext, filename
#                 yield open_name
                f.close()
            except:
                f.close()
                pass

In [6]:
data = []

In [7]:
for i in read_txt('/Users/Valeriya/Desktop/Interfax/Анонс_нерасп','Анонс'):
    data.append(i)
#     with open('anonce.txt', 'a') as f:
#         f.write(re.findall('[0-9]_(.*)', i)[0].strip('.html'))
#         f.write('\n')

In [8]:
for i in read_txt('/Users/Valeriya/Desktop/Interfax/Анонс_расп', 'Анонс'):
    data.append(i)
    
# #     with open('anonce.txt', 'a') as f:
# #         f.write(re.findall('[0-9]_(.*)', i)[0].strip('.html'))
# #         f.write('\n')

In [9]:
for i in read_txt('/Users/Valeriya/Desktop/Interfax/Остальные_жанры', 'Остальное'):
    data.append(i)
    
#     with open('other.txt', 'a') as f:
#         f.write(re.findall('[0-9]_(.*)', i)[0].strip('.html'))
#         f.write('\n')

In [10]:
for i in tqdm.tqdm_notebook(read_txt('/Users/Valeriya/Desktop/Interfax/15000_other_genres', 'Остальное')):
    data.append(i)

In [11]:
# len(data)

In [12]:
df = pd.DataFrame(data, columns=['y', 'X', 'filename'])

In [13]:
len(df.filename)

15401

In [14]:
num = 0

In [15]:
features_arr = []

In [16]:
path1 = '/Users/Valeriya/Desktop/Interfax/all_features/features_15000_other_genres/'
path2 = '/Users/Valeriya/Desktop/Interfax/features_all_22november/'
curr_path = ''
for i in tqdm.tqdm_notebook(df.filename):
    if i.endswith('html'):
        fromdf = i.strip('.html')
        fromdf = re.sub('[0-9]+_', '', fromdf)
    elif i.endswith('xml'):
        fromdf = i.strip('.xml')
        fromdf = re.sub('[0-9]+_', '', fromdf)
        
    name = 'dateFeature_'+fromdf+'.txt'
    
    if os.path.exists(path2 + name):
        curr_path = path2
    elif os.path.exists(path1+name):
        curr_path = path1
    else:
        features_arr.append(None)
        continue
        
#         print('ok')
    f = open(curr_path+name)
    file = f.readlines()
    curr_vect = []
    for row in file:
        row_vect = []
        nums = row.split('|')[0].split('\t')
        for num in nums:
            num = num.strip('\ufeff').strip()
#                 print(num)
            try:
                row_vect.append(int(num))
            except:
#                     print(num)
                if num == '':
                    row_vect.append(None)
        curr_vect.append(row_vect)
#     f.close()
    features_arr.append(curr_vect)
    f.close()
#             features_arr.append(nums)

In [17]:
len(features_arr)

15401

In [18]:
df['form_feat'] = features_arr

In [19]:
# df

In [20]:
formal_annonce = df[df.y == 'Анонс'].form_feat
formal_other = df[df.y == 'Остальное'].form_feat

In [21]:
# formal_annonce

In [23]:
# # 1st number from rows
# fst_nums_annonce = []
# for i in formal_annonce:
#     try:
#         for arr in i:
#             if arr[0] is not None:
#                 if arr[0] == 833579:
#                     print('!')
#                 fst_nums_annonce.append(arr[0])
            
#     except:
#         pass

In [24]:
# fst_nums_other = []
# for i in enumerate(formal_other):
#     try:
#         for arr in i[1]:
#             if arr[0] is not None:
#                 if arr[0] == 833579:
#                     print(i[0])
#                 fst_nums_other.append(arr[0])
            
#     except:
#         pass

In [22]:
fst_nums_annonce = []

In [23]:
for index, row in df[df.y == 'Анонс'].iterrows():
    ok_row = row['form_feat']
    if ok_row is not None:
        for arr in ok_row:
            try:
                if arr[0] is not None:
                    fst_nums_annonce.append(arr[0])
            except:
                pass

In [24]:
# fst_nums_annonce

In [25]:
fst_nums_other = []
for index, row in df[df.y == 'Остальное'].iterrows():
    ok_row = row['form_feat']
    if ok_row is not None:
        for arr in ok_row:
            try:
                if arr[0] is not None:
                    if arr[0] == 833579:
                        print(index)
                    fst_nums_other.append(arr[0])
            except:
                pass

In [26]:
import numpy as np

In [27]:
# df.iloc[2412]

In [28]:
np.mean(fst_nums_annonce), np.median(fst_nums_annonce), np.max(fst_nums_annonce), np.min(fst_nums_annonce)

(400.63982626127631, 5.0, 14102, 1)

In [29]:
np.mean(fst_nums_other), np.median(fst_nums_other), np.max(fst_nums_other), np.min(fst_nums_other)

(218.15554922082859, 66.0, 328569, 1)

In [30]:
last_nums_annonce = []
for index, row in df[df.y == 'Анонс'].iterrows():
    ok_row = row['form_feat']
    if ok_row is not None:
        for arr in ok_row:
            try:
                if arr[1] is not None:
                    last_nums_annonce.append(arr[1])
            except:
                pass

In [31]:
# last_nums_annonce

In [32]:
last_nums_other = []
for index, row in df[df.y == 'Остальное'].iterrows():
    ok_row = row['form_feat']
    if ok_row is not None:
        for arr in ok_row:
            try:
                if arr[1] is not None:
                    last_nums_other.append(arr[1])
            except:
                pass

In [33]:
# last_nums_other

In [34]:
np.mean(last_nums_annonce), np.median(last_nums_annonce), np.max(last_nums_annonce), np.min(last_nums_annonce)

(267.28967591045773, 0.0, 329083, 0)

In [35]:
np.mean(last_nums_other), np.median(last_nums_other), np.max(last_nums_other), np.min(last_nums_other)

(221.39604713036869, 30.0, 329434, 0)

In [75]:
# mean, median, max по данному тексту

## Добавляем новые признаки

In [36]:
new_features = []

for row in df.form_feat:
    try:
        nums = []
        for item in row:
            nums.append(item[0])
        new_features.append([np.mean(nums), np.median(nums), np.max(nums)])
    except:
        new_features.append(None)
        

In [37]:
df['new_features'] = new_features

In [38]:
new_df = df.dropna()

In [39]:
new_df.y.value_counts()

Остальное    5023
Анонс         155
Name: y, dtype: int64

In [40]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

In [41]:
X = new_df[['X', 'new_features']]
y = new_df['y']

In [42]:
count_vect = CountVectorizer()
X_2 = count_vect.fit_transform(X.X)
tf_transformer = TfidfTransformer()
X_2 = tf_transformer.fit_transform(X_2)

In [43]:
# X_test_Xs = count_vect.transform(X_test.X)
# X_test_Xs = tf_transformer.transform(X_test_Xs)

In [44]:
X_selected = SelectKBest(chi2, k=1000).fit_transform(X_2, y)

In [45]:
from scipy.sparse import hstack
from scipy.sparse import csr_matrix
# list(X_train.new_features)

In [46]:
# добавим наши формальные признаки
X_final = np.append(X_selected.todense(), list(X.new_features), axis=1)

In [47]:
X_train, X_test, y_train, y_test = train_test_split(X_final, y, test_size=0.3, stratify=y)

In [125]:
# from sklearn.decomposition import PCA

In [115]:
# pca = PCA(n_components=20)

In [116]:
# X_pca_tr= pca.fit(X_train_final)

In [48]:
model = XGBClassifier(max_depth=3, n_estimators=1500)

In [49]:
model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=1500, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [50]:
y_pred = model.predict(X_test)

In [54]:
print(classification_report(y_test, list(y_pred)))

             precision    recall  f1-score   support

      Анонс       1.00      0.89      0.94        47
  Остальное       1.00      1.00      1.00      1507

avg / total       1.00      1.00      1.00      1554

